In [ ]:


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

def scorer(estimator, X, y):
    return -log_loss(y, estimator.predict_proba(X))

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
df=pd.read_json('../input/train.json')
df['priceperbed']=(df['price'].clip(upper=7000)/df['bedrooms'].clip(lower=1))
df['created']=df['created'].astype(np.datetime64)
df['created_day']=np.array(df.created.values, dtype='datetime64[D]').astype(np.float32)%7
df['created_week']=np.array(df.created.values, dtype='datetime64[W]').astype(np.float32)
df['created_hour']=np.array(df.created.values, dtype='datetime64[h]').astype(np.float32)%24
df['desc_count']=df.description.apply(lambda x: len(x.split())).clip(upper=150)
df['features_count']=df.features.apply(lambda x: len(x))
df['photos_count']=df.photos.apply(lambda x: len(x))


In [ ]:
df['features']=df['features'].apply(lambda x: list(map(str.lower, x)))
df_features=pd.DataFrame(np.hstack(df.features.values), columns=['feature'])
df_feature_counts=pd.DataFrame(df_features.groupby(['feature']).size(), 
                               columns=['size']).reset_index()
df_feature_counts=df_feature_counts.sort_values('size', ascending=False)
feature_list=df_feature_counts.loc[df_feature_counts['size']>5000]['feature'].values
feature_list

In [ ]:
for feature in feature_list:
        df[feature]=df['features'].apply(lambda x: feature in x)

In [ ]:
df_tv, df_test = train_test_split(df, random_state=0)
df_train, df_val = train_test_split(df_tv, random_state=0)

In [ ]:
cols=['price', 'bathrooms', 'bedrooms', 'priceperbed', 'created_hour', 'desc_count', 'photos_count', 'features_count', 
      'elevator', 'hardwood floors', 'cats allowed', 'dogs allowed', 'doorman', 'dishwasher', 
      'laundry in building', 'no fee', 'fitness center', 'laundry in unit', 'pre-war', 'roof deck', 
      'outdoor space', 'dining room']

In [ ]:
cols=['price', 'bathrooms', 'bedrooms', 'priceperbed','created_hour',  'desc_count', 'photos_count',
'features_count', 'no fee', 'hardwood floors', 'laundry in building']


In [ ]:
    clf=ExtraTreesClassifier(max_depth=23, n_estimators=1000,
                             min_samples_split=10, random_state=0) 
    clf.fit(df_train[cols].values, df_train['interest_level'])
    y_pred=clf.predict_proba(df_train[cols])
    score=log_loss(df_train['interest_level'].values, y_pred)
    y_pred=clf.predict_proba(df_val[cols])
    score2=log_loss(df_val['interest_level'].values, y_pred)
    y_pred=clf.predict_proba(df_test[cols])
    score3=log_loss(df_test['interest_level'].values, y_pred)
    print("%.6f %.6f %.6f"%(score, score2, score3))

In [ ]:
    import xgboost as xgb
    
    param = {}
    param['objective'] = 'multi:softprob'
    #param['eta'] = 0.1
    #param['max_depth'] = 23
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    #param['min_child_weight'] = 10
    #param['subsample'] = 0.7
    #param['colsample_bytree'] = 0.7
    param['seed'] = 0
    plst = list(param.items())
    target_num_map = {'high':0, 'low':1, 'medium':2}
    train_y = np.array(df_train['interest_level'].apply(lambda x: target_num_map[x]))
    xgtrain = xgb.DMatrix(df_train[cols].values, label=train_y)
    model = xgb.train(plst, xgtrain, 100)
    
    xgtest = xgb.DMatrix(df_train[cols].values)
    y_pred=model.predict(xgtest)
    score=log_loss(df_train['interest_level'].values, y_pred)
    xgtest = xgb.DMatrix(df_val[cols].values)
    y_pred=model.predict(xgtest)
    score2=log_loss(df_val['interest_level'].values, y_pred)
    xgtest = xgb.DMatrix(df_test[cols].values)
    y_pred=model.predict(xgtest)
    score3=log_loss(df_test['interest_level'].values, y_pred)
    print("%.6f %.6f %.6f"%(score, score2, score3))

In [ ]:
import operator
sorted(dict(zip(cols, clf.feature_importances_)).items(), key=operator.itemgetter(1))

In [ ]:
feature_list

In [ ]:
df['priceperbed']=(df['price'].clip(upper=7000)/df['bedrooms'].clip(lower=1))
df['created_month']=np.array(df.created.values, dtype='datetime64[M]').astype(np.float32)


In [ ]:
df['created_week'].max()

In [ ]:
df['laundry in unit']=(df['laundry in unit'])|(df['laundry in building'])

In [ ]:
df['laundry in unit'].sum()